In [16]:
import pandas as pd
import re
import folium

df = pd.read_csv("20250304_data.csv", encoding='utf-8')


Convert DMS Coordinates to Decimal



In [17]:
def dms_to_decimal(dms_str):
    """Convert DMS (Degrees, Minutes, Seconds) to Decimal Degrees."""
    match = re.match(r"(\d+)°(\d+)'(\d+)\"([NSEW])", dms_str)
    if not match:
        return None  # Handle bad data gracefully

    degrees, minutes, seconds, direction = match.groups()
    decimal = float(degrees) + float(minutes) / 60 + float(seconds) / 3600

    # Make it negative for South or West
    if direction in "SW":
        decimal = -decimal

    return round(decimal, 6)  # Round for accuracy

# Apply function to the DataFrame
df["lat_dec"] = df["lat"].apply(dms_to_decimal)
df["long_dec"] = df["long"].apply(dms_to_decimal)

# Display results
df.head()

,name,date,OS,lat,long,altitude,lat_dec,long_dec,Image
0,Ruin past the White House on the Rubha Dunan p...,23/04/2024,NC 02440 07127,"58°0'33""N","5°20'44""W",19,58.009167,-5.345556,https://github.com/MorningGlass/morningglass.g...


Create a map

In [ ]:
# Create a Folium map centered on an approximate middle point
map_center = [df["lat_dec"].mean(), df["long_dec"].mean()]
# m = folium.Map(location=map_center, zoom_start=11)
m = folium.Map(location=map_center, zoom_start=11, tiles="OpenStreetMap")

# Add markers for each ruin
for _, row in df.iterrows():
    # HTML & CSS for styled popup
    popup_html = f"""
    <div style="
        font-family: Helvetica, sans-serif;
        font-size: 14px;
        color: black;
        padding: 10px;
        text-align: left;
        background-color: #F7F8FA;
        max-width: 90vw; /* Make popup width responsive */
        word-wrap: break-word;

    ">
        <h4 style="font-size: 18px; margin: 5px 0; color: black;">{row['name']}</h4>
        <p><b>OS Grid Ref:</b> {row['OS']}</p>
        <p><b>Location: </b>{row['lat']}, {row['long']}</p>
        <p><b>Date photographed</b>: {row['date']}</p>
        <p><b>Altitude:</b> {row['altitude']}m</p>
        <img src="{row['Image']}" alt="Image of {row['name']}" style="width: 100%; height: auto; border-radius: 5px;">
    </div>
    """

    # Add marker with custom popup
    folium.Marker(
        location=[row["lat_dec"], row["long_dec"]],
        popup=folium.Popup(popup_html, max_width=400),  # Controls popup size
        tooltip=row["name"],  # Shows the name on hover
    ).add_to(m)

# Save and display map
m.save("map.html")
m